In [1]:
import cv2
import numpy as np
import math

In [110]:
imgc = cv2.imread("hough.jpg")
gim = cv2.imread("hough.jpg",0)

In [3]:
def Pad(img):
    Ix=list()
    for line in img:
        line = list(line)
        line.insert(0,0)        #Column padding to perform convolution
        line.insert(len(line),0)
        Ix.append(line)
    Opx = list()
    RowPad = [ 0 for i in range(len(Ix[0])) ] #Row padding to perform convolution
    Opx.append(RowPad)
    for i in Ix:
        Opx.append(i)
    Opx.append(RowPad)    
    Opx = np.asarray(Opx)
    return Opx

In [4]:
N = gim.shape[0]
M = gim.shape[1]
Img_padx = Pad(gim)  #Pad Image
Img_pady = Img_padx

In [5]:
nx = [[1,0,-1],
      [2,0,-2],
      [1,0,-1]]  #Sobel X kernel

ny = [[1,2,1],
      [0,0,0],
      [-1,-2,-1]] #Sobel Y kernel
#Flip both kernels horizontally and vertically 
for i in nx:
        temp = i[0]
        i[0] = i[2]
        i[2] = temp
        
temp = ny[0]
ny[0] = ny[2]
ny[2] = temp

In [6]:
sobelxImage = [[0]*M for i in range(N)]
sobelxImage = np.asarray(sobelxImage)
sobelyImage = [[0]*M for i in range(N)]
sobelyImage = np.asarray(sobelyImage)
    
imagex = Img_padx
for i in range(1,N):     # Loop over every pixel of the image
    for j in range(1,M):
        x = imagex[i-1:i+2,j-1:j+2]
        result = [[a*b for a, b in zip(k, l)] for k, l in zip(x, nx)] #Element wise Multiplication
        elex = 0
        for row in result:
            for ele in row: 
                elex = elex+ele  #Addition of the products to obtain value for that pixel
        
            sobelxImage[i-1][j-1] = elex    
            
        imagey = Img_pady
        y = imagey[i-1:i+2,j-1:j+2]
        result = [[a*b for a, b in zip(k, l)] for k, l in zip(y, ny)] #Element wise Multiplication
        eley = 0
        for row in result:
            for ele in row: 
                eley = eley+ele  #Addition of the products to obtain value for that pixel
        sobelyImage[i-1][j-1] = eley

In [ ]:
cv2.imwrite("ConvX_image.png",sobelxImage)

In [7]:
def Normalize(Img_conv): #Function to normalize the image to remove negative values and have the values between 0 and 1.
    Opx_norm = Img_conv
    maxval = 0
    for i in Img_conv:
        for j in i:
            if maxval<abs(j):
                maxval = j
    print(maxval)
    for i in range(0,Img_conv.shape[0]):
        for j in range(0,Img_conv.shape[1]):
            Opx_norm[i][j] =  abs(Img_conv[i][j])*255/abs(maxval)
    return Opx_norm

In [8]:
Norx = Normalize(sobelyImage)
Nory = Normalize(sobelxImage)

751
719


In [17]:
def find_edges(img):
    eximg = np.zeros((img.shape[0],img.shape[1]))
    for i in range(N):
        for j in range(M):
            if img[i,j] > 50:
                eximg[i,j] = 255
    return eximg            

In [18]:
edx = find_edges(Norx)
edy = find_edges(Nory)
ed = edx + edy

True

In [96]:
def find_cirpoint(theta,x,y):  #Calculate centre
    a = int(y - R * math.cos(math.radians(theta)))
    b = int(x - R * math.sin(math.radians(theta)))
    return a,b
    

In [97]:
def cal_circle(x,y): #Draw circle for each pixel value
    newcen = []
    for i in range(0,360):
        d,f = find_cirpoint(i,x,y)
        newcen.append([d,f])
    return newcen   

In [98]:
R = 22  #Radius
accu = np.zeros((2*N,2*M)) #Accumulator Matrix

In [99]:
for i in range(N):   #Building Accumulator Matrix
    for j in range(M):
        if ed[i,j] == 255:
            pixcen = cal_circle(i,j)
            for k in range(len(pixcen)):
                accu[pixcen[k][0],pixcen[k][1]] = accu[pixcen[k][0],pixcen[k][1]] + 1 

In [100]:
def max_indices(arr, k):  #Get Peak Occurences
    assert k <= arr.size,'k should be smaller or equal to the array size' 
    arr_ = arr.astype(float)  
    max_idxs = []
    for _ in range(k):
        max_element = np.max(arr_)
        if np.isinf(max_element):
            break
        else:
            idx = np.where(arr_ == max_element)
        max_idxs.append(idx)
        arr_[idx] = -np.inf
    return np.asarray(max_idxs)

In [111]:
lp = max_indices(accu,40)

In [112]:
peaks = []
for i in range(lp.shape[0]):
    n = lp[i][0].shape[0]
    if n>1:
        for j in range(n):
            peaks.append([lp[i][0][j], lp[i][1][j]])
    else:
        peaks.append([lp[i][0][0], lp[i][1][0]])

In [113]:
for i,j in peaks: #Plot Circles for coins
    imgc = cv2.circle(imgc,(i,j),R,(0,255,0))

In [114]:
cv2.imwrite("coin.jpg",imgc)

True

In [117]:
cv2.imwrite("accub.jpg",accu)

True